In [1]:
from mindquantum.core.circuit import Circuit, UN
from mindquantum.core.gates import H, ZZ, RX, XX, YY, RY, X
from mindquantum.core.operators import Hamiltonian, QubitOperator
from mindquantum.framework import MQAnsatzOnlyLayer
from mindquantum.simulator import inner_product, Simulator
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz
from mindquantum.core.circuit import shift, Circuit
from mindquantum.core.operators import TimeEvolution, QubitOperator
from mindspore.common.parameter import Parameter

import matplotlib.pyplot as plt
import networkx as nx
import mindspore.nn as nn
import mindspore as ms
import numpy as np  
import random
import pandas as pd

qubit_num = 8

# 定义问题哈密顿量及问题层线路circuit1

In [2]:
def Construct_Hamiltonian_C():
    H_C = QubitOperator() 
    H_C += QubitOperator('',18)+ QubitOperator('Z0', -0.5) + QubitOperator('Z1', -4.5)+ QubitOperator('Z2', -2.5)+ QubitOperator('Z3', -1.5)+ QubitOperator('Z4', -2) + QubitOperator('Z5', -2.5)+ QubitOperator('Z6', -2.5)+ QubitOperator('Z7', -2)
    
    return H_C

In [3]:
H_C = Construct_Hamiltonian_C()
print (H_C)

18 [] +
-1/2 [Z0] +
-9/2 [Z1] +
-5/2 [Z2] +
-3/2 [Z3] +
-2 [Z4] +
-5/2 [Z5] +
-5/2 [Z6] +
-2 [Z7] 


In [4]:
hc = H_C - 18*QubitOperator(())
print (hc)

-1/2 [Z0] +
-9/2 [Z1] +
-5/2 [Z2] +
-3/2 [Z3] +
-2 [Z4] +
-5/2 [Z5] +
-5/2 [Z6] +
-2 [Z7] 


In [5]:
# circuit1是问题哈密顿量H_C中去掉单位算子I构造的电路，属于问题层(根据目标函数构建)
circuit1 = TimeEvolution(hc,{'gamma':0.5}).circuit
circuit1.svg()

# 构造混合层线路𝑒^(−𝑖𝛽𝐻_𝑀 )

In [6]:
def build_hb(para):              #自定义一个电路构建块hc（目标哈密顿量）， # 
    hb = Circuit()               # 创建量子线路
    
    hb += RX({para:2}).on(0, [2,3,6])  
    
    hb += RX({para:2}).on(1, [6,7])
    
    hb += RX({para:2}).on(2, [0,3,7])
    
    hb += RX({para:2}).on(3, [0,2,5])
    
    hb += RX({para:2}).on(4, [5,6])
    
    hb += RX({para:2}).on(5, [3,4,7,6])
    
    hb += RX({para:2}).on(6, [0,1,4,5])
    
    hb += RX({para:2}).on(7, [1,2,5])
    
#    hb.barrier()                    # 添加Barrier以方便展示线路
    return hb

In [7]:
# pylint: disable=W0104
circuit2 = build_hb('beta')
circuit2.svg()

# 总线路ansatz

In [8]:
def build_ansatz(p):                    # g是max-cut问题的图，p是ansatz线路的层数                      
    circ = UN(X, qubit_num)             # 制备初始态，创建量子线路
    for i in range(p):
        circ += TimeEvolution(hc, {f'γ{i}':0.5}).circuit       # 问题层H_C  （依目标函数而设计）
        circ += build_hb(f'β{i}')       # 混合层   （依约束条件1而设计）
        
    return circ

In [9]:
p = 1
build_ansatz(p).svg()                                      #展示完整的量子线路
#build_ansatz(p).svg().to_file(filename='Full_circuit.svg')   #展示完整的量子线路

# 单次实验

In [10]:
p = 8
ham = Hamiltonian(Construct_Hamiltonian_C()) # 生成哈密顿量
circ = build_ansatz(p)                  # 生成ansatz线路

ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
sim = Simulator('mqvector', circ.n_qubits)                     # 创建模拟器，backend使用‘mqvector’，能模拟5个比特（'circ'线路中包含的比特数）
grad_ops = sim.get_expectation_with_grad(ham, circ)            # 获取计算变分量子线路的期望值和梯度的算子
net = MQAnsatzOnlyLayer(grad_ops)                              # 生成待训练的神经网络
net.weight = Parameter((np.random.rand(len(net.weight))*np.pi).astype(np.float32))
Initial_parameter = dict(zip(circ.params_name, net.weight.asnumpy()))
opti = nn.Adam(net.trainable_params(), learning_rate = 0.05)     # 设置针对网络中所有可训练参数、学习率为0.05的Adam优化器
train_net = nn.TrainOneStepCell(net, opti)                     # 对神经网络进行一步训练                   # 对神经网络进行一步训练

for i in range(500):
    energy = train_net()    # 将神经网络训练一步并计算得到的结果。注意：每当'train_net()'运行一次，神经网络就训练了一步
    
    if i%10 == 0:
        print('i = %d energy = %f'%(i,energy))
    

i = 0 energy = 24.780571
i = 10 energy = 19.697283
i = 20 energy = 18.540741
i = 30 energy = 17.459238
i = 40 energy = 17.337276
i = 50 energy = 17.243378
i = 60 energy = 17.225019
i = 70 energy = 17.220100
i = 80 energy = 17.212917
i = 90 energy = 17.210323
i = 100 energy = 17.208990
i = 110 energy = 17.208456
i = 120 energy = 17.208202
i = 130 energy = 17.208084
i = 140 energy = 17.208008
i = 150 energy = 17.207962
i = 160 energy = 17.207928
i = 170 energy = 17.207903
i = 180 energy = 17.207884
i = 190 energy = 17.207865
i = 200 energy = 17.207850
i = 210 energy = 17.207832
i = 220 energy = 17.207817
i = 230 energy = 17.207800
i = 240 energy = 17.207785
i = 250 energy = 17.207769
i = 260 energy = 17.207775
i = 270 energy = 17.368275
i = 280 energy = 17.211460
i = 290 energy = 17.217737
i = 300 energy = 17.214418
i = 310 energy = 17.210033
i = 320 energy = 17.208481
i = 330 energy = 17.208012
i = 340 energy = 17.207783
i = 350 energy = 17.207678
i = 360 energy = 17.207684
i = 370 ener

In [11]:
pr = dict(zip(circ.params_name, net.weight.asnumpy())) # 获取线路参数
state = circ.get_qs(pr=pr)

v1 = [0]* 2**qubit_num
v1[217] = 1       #这里指定的量子态v也是降序的qubits，即应该是q2q1q0,即应该是110，化为十进制是1*2^2+1*2^1+0*2^0=6
fid1 = np.abs(np.vdot(v1,state))**2 

state = circ.get_qs(pr=pr,ket=True)
#print('untrained quantum states:{}'.format(state)) #检查输出态是否满足可行解空间
print('initial parameter is: {}'.format(Initial_parameter))   #检查初始参数是否是从（0,π选的）

print(('optimal parameter: {}'.format(pr)))
print('fid1: {}'.format(fid1)) #获得解的概率
print('The F value is: {}'.format(energy))
print('A.R.: {}'.format(17/energy))  #近似比

initial parameter is: {'γ0': 1.0903709, 'β0': 1.5418909, 'γ1': 1.2817658, 'β1': 0.28450298, 'γ2': 2.5300527, 'β2': 1.6403288, 'γ3': 1.211785, 'β3': 0.7417103, 'γ4': 3.0469055, 'β4': 2.5254414, 'γ5': 1.1109836, 'β5': 2.8021004, 'γ6': 1.3640991, 'β6': 0.46849418, 'γ7': 1.2425755, 'β7': 2.364849}
optimal parameter: {'γ0': 1.0903766, 'β0': 1.5947242, 'γ1': 2.2222729, 'β1': 0.068663955, 'γ2': 2.3772597, 'β2': 1.4897078, 'γ3': 0.9747612, 'β3': 0.37260997, 'γ4': 3.1614003, 'β4': 2.7544684, 'γ5': 1.3899249, 'β5': 3.1345139, 'γ6': 1.1005232, 'β6': 0.35549337, 'γ7': 1.3102983, 'β7': 3.4937952}
fid1: 0.9690042578899195
The F value is: [17.208181]
A.R.: [0.98790216]


In [12]:
circ.measure_all()                               # 为线路中所有比特添加测量门
sim.sampling(circ, pr=pr, shots=1000).svg()

# 在不同的初始化参数下多次训练线路，以Excel（.xlsx）文件的形式输出训练结果

In [ ]:
if __name__ == "__main__":
    #创建一个空的DataFrame来存储结果
    results = []

    for p in range(1, 9):
        print('The p value is: {}'.format(p))
        for t in range(1, 501):
            ham = Hamiltonian(Construct_Hamiltonian_C()) # 生成哈密顿量
            circ = build_ansatz(p)                  # 生成ansatz线路

            ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
            sim = Simulator('mqvector', circ.n_qubits)                     # 创建模拟器，backend使用‘mqvector’，能模拟5个比特（'circ'线路中包含的比特数）
            grad_ops = sim.get_expectation_with_grad(ham, circ)            # 获取计算变分量子线路的期望值和梯度的算子
            net = MQAnsatzOnlyLayer(grad_ops)                              # 生成待训练的神经网络
            net.weight = Parameter((np.random.rand(len(net.weight)) * np.pi).astype(np.float32))
            theta = dict(zip(circ.params_name, net.weight.asnumpy()))

            opti = nn.Adam(net.trainable_params(), learning_rate = 0.05)     # 设置针对网络中所有可训练参数、学习率为0.05的Adam优化器
            train_net = nn.TrainOneStepCell(net, opti)                     # 对神经网络进行一步训练                   
                
            
            for i in range(500):
                energy = train_net()
                #if i%500 == 0:
                #    print('i = %d energy = %f'%(i,energy))
                
            pr = dict(zip(circ.params_name, net.weight.asnumpy())) 
            state = circ.get_qs(pr=pr)
            
            v1 = [0]* 2**qubit_num
            v1[217] = 1       #这里指定的量子态v也是降序的qubits，即应该是q2q1q0,即应该是110，化为十进制是1*2^2+1*2^1+0*2^0=6
            fid1 = np.abs(np.vdot(v1,state))**2 

            # 将结果添加到列表
            results.append({"p": p, "t": t, "energy": energy, "A.R.": 17/energy, "Fidelity": fid1})
            
            #命令交互行输出，便于区分（实际上没啥必要）            
            #print('t: {}'.format(t))

        # 将列表转换为DataFrame
        df = pd.DataFrame(results)

        # 将DataFrame写入Excel文件
        df.to_excel(r"/home/lzztlb/liguanghui/AAResearch/liguanghui（code)/科研/MWVC/MWVCv2_/example/QAOA+ data/example.xlsx", index=False)
    